In [1]:
import os
os.chdir("/mnt/volumes/ss-sai-bd-ga/wuzhenyan/tool_script")

In [2]:
DATE = "0919"

In [3]:
import json
from utils.search_utils.es_client import ElasticSearchClient
from search.qa_bot.qwen3_reranker.meta import *


class SearchData:

    def __init__(self):
        self.es_client = ElasticSearchClient(CLUSTER_NAME)

    def search(self, text: str, log_size: int = 100, qa_size: int = 10):
        # 查询日志数据
        log_query_dsl_dict = self.log_query_recall_strategy(text, log_size)
        log_item_list = self.es_client.search(log_query_dsl_dict, LOG_INDEX_NAME, need_parse=True)
        # 查询qa数据
        qa_query_dsl_dict = self.qa_query_recall_strategy(text, qa_size)
        qa_item_list = self.es_client.search(qa_query_dsl_dict, QA_INDEX_NAME, need_parse=True)
        # 拼装结果
        result_dict = {
            LOG_INDEX_NAME: log_item_list,
            QA_INDEX_NAME: qa_item_list,
        }
        return result_dict

    def log_query_recall_strategy(self, text, size):
        """日志query召回策略"""
        query_dsl_dict = {
            "query": {
                "bool": {
                    "should": [
                        {"match": {"text": {"query": text, "boost": 1.0}}},
                        {"match": {"text_rewrite": {"query": text, "boost": 1.0}}},
                    ],
                    "minimum_should_match": 1
                }
            },
            "_source": ["text", "text_rewrite", "pv"],
            "size": size
        }
        return query_dsl_dict

    def qa_query_recall_strategy(self, text, size):
        """问答库query召回策略"""
        query_dsl_dict = {
            "query": {
                "bool": {
                    "should": [
                        {"match": {"text": {"query": text, "boost": 1.0}}},
                        {"match": {"text_alias": {"query": text, "boost": 1.0}}},
                    ],
                    "minimum_should_match": 1
                }
            },
            "_source": ["text", "text_alias", "pv"],
            "size": size
        }
        return query_dsl_dict

In [4]:
# Load Data
import pandas as pd

neg_df = pd.read_csv(f"data/cloud_share/qabot_relevance_data/qwen3_reranker/active_learning/{DATE}/negative_seed.tsv", sep='\t')
neg_query_list = neg_df["query"].to_list()

pos_df = pd.read_csv(f"data/cloud_share/qabot_relevance_data/qwen3_reranker/active_learning/{DATE}/positive_seed.tsv", sep='\t')
pos_query_list = pos_df["query"].to_list()

In [5]:
obj = SearchData()
new_query_list = []
score_list = []
text_list = []
old_query_list = []
for i in range(len(neg_query_list)):
    query = neg_query_list[i]
    text = neg_df["title"][i]
    result_dict = obj.search(query, log_size=10, qa_size=200)["qa_bot_log_data_v1"]
    for j in range(len(result_dict)):
        new_query = result_dict[j]["text"]
        if new_query != query and result_dict[j]["feature"]["recall_score"]>50:
            new_query_list.append(new_query)
            text_list.append(text)
            score_list.append(result_dict[j]["feature"]["recall_score"])
            old_query_list.append(query)

new_neg_df = pd.DataFrame({
    "old_query": old_query_list,
    "query":new_query_list,
    "text":text_list,
    "label": ["no"]*len(text_list),
    "recall_score": score_list
})

In [ ]:
new_query_list = []
text_list = []
score_list = []
old_query_list = []

for i in range(len(pos_query_list)):
    query = pos_query_list[i]
    text = pos_df["title"][i]
    result_dict = obj.search(query, log_size=10, qa_size=200)["qa_bot_log_data_v1"]
    for j in range(len(result_dict)):
        new_query = result_dict[j]["text"]
        score = result_dict[j]["feature"]["recall_score"]
        if new_query != query and score>70 and ("&" not in new_query):
            new_query_list.append(new_query)
            text_list.append(text)
            score_list.append(score)
            old_query_list.append(query)

new_pos_df = pd.DataFrame({
    "old_query": old_query_list,
    "query":new_query_list,
    "text":text_list,
    "label": ["yes"]*len(text_list),
    "recall_score": score_list
})

In [7]:
# 相关性模型过滤neg sample
from search.qa_bot.service_bot_intent.step3_onnx_infer import onnx_inference
from transformers import BertTokenizerFast
import onnxruntime as ort
import torch.nn as nn
import numpy as np
import torch
base_model_dir = "data/cloud_share/qabot_miner/bert_model/model/base_model/bert-base-chinese" 
onnx_path = "data/cloud_share/qabot_relevance_model/model.onnx"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def compute_similarity(df):
    text_list = []
    for i in range(len(df)):
        user_query = df["query"][i]
        question = df["text"][i]
        user_query = user_query.lower()
        user_query = user_query.replace('的', '')
        user_query = user_query.replace('啊', '')
        user_query = user_query.replace('呀', '')
        user_query = user_query.replace('我', '')
        user_query = user_query.replace('哎', '')
        user_query = user_query.replace('唉', '')
        user_query = user_query.replace('流量信息', '流量')
        text_list.append(f"{user_query}[SEP]{question}")

    text_batch = {
                        "texts": text_list
                    }

    tokenizer = BertTokenizerFast.from_pretrained(base_model_dir, do_lower_case=True)
    train_text_encodings = tokenizer(text_batch["texts"], padding=True,
                                        truncation=True, max_length=128, return_tensors="pt")

    train_text_encodings = train_text_encodings.to(device)
    input_ids = train_text_encodings['input_ids'].cpu().numpy()
    attention_mask = train_text_encodings['attention_mask'].cpu().numpy()
    token_type_ids = train_text_encodings['token_type_ids'].cpu().numpy()
    sess = ort.InferenceSession(onnx_path, None)  # 加载onnx模型
    onnx_output = sess.run(['output'],
                            {'input_ids': input_ids,
                            'attention_mask': attention_mask,
                            "token_type_ids": token_type_ids
                            })
    logits = torch.tensor(np.array(onnx_output))[0].to(device)
    softmax = nn.Softmax(dim=-1)
    prob = softmax(logits)
    similarity_tensor = torch.narrow(prob, 1, 1, 1)
    similarity_tensor = torch.squeeze(similarity_tensor, 1).cpu().numpy()
    return similarity_tensor

/mnt/volumes/ss-sai-bd-ga/wuzhenyan/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
similarity_tensor_pos = compute_similarity(new_pos_df)
similarity_tensor_neg = compute_similarity(new_neg_df)

In [9]:
new_neg_df["score"] = similarity_tensor_neg
new_pos_df["score"] = similarity_tensor_pos

In [10]:
# new_neg_df.to_csv(f"data/cloud_share/qabot_relevance_data/qwen3_reranker/active_learning/{DATE}/negative_sample.tsv", sep='\t', index=False)
# new_pos_df.to_csv(f"data/cloud_share/qabot_relevance_data/qwen3_reranker/active_learning/{DATE}/postive_sample.tsv", sep='\t', index=False)

In [11]:
# new_neg_df[similarity_tensor_neg<0.4][["query", "text", "label"]].to_csv(f"data/cloud_share/qabot_relevance_data/qwen3_reranker/active_learning/{DATE}/negative_sample.tsv", sep='\t', index=False)
# new_pos_df[similarity_tensor_pos>0.6][["query", "text", "label"]].to_csv(f"data/cloud_share/qabot_relevance_data/qwen3_reranker/active_learning/{DATE}/postive_sample.tsv", sep='\t', index=False)

In [12]:
# 统计覆盖率
# LLM生成未覆盖的query
import time
import json
from tqdm import tqdm
from utils.llm_utils.serverless_function import request_llm
SYSTEM_PROMPT = """你需要对给出的query进行改写，不能改变原有query的语义。输出5条改写的query，以json列表的格式输出"""
count = 0
non_test_list = []
new_query_list = []

for i in tqdm(range(len(neg_df))):
    query = neg_df["query"][i]
    text = neg_df["title"][i]
    if text in new_neg_df[similarity_tensor_neg<0.4]["text"].to_list():
        count += 1
    else:
        USER_PROMPT = f"query: {query}"
        history = [
            SYSTEM_PROMPT,
            USER_PROMPT
        ]
        res = request_llm(history=history)
        try:
            res_query_list = json.loads(res[1]["choices"][0]["message"]["content"].replace("```json","").replace("```","").replace("\n",""))
            non_test_list.append(text)
            new_query_list.append(res_query_list)
        except Exception as e:
            print(e)

        
print(count/len(neg_df))
llm_texts = []
llm_new_queries = []
for i in range(len(new_query_list)):
    for q in new_query_list[i]:
        q = q.replace("query:", "").strip()
        llm_new_queries.append(q)
        llm_texts.append(non_test_list[i])
llm_neg = pd.DataFrame({
    "query":llm_new_queries,
    "text":llm_texts,
    "label":["no"]*len(llm_texts)
})


100%|██████████| 80/80 [01:23<00:00,  1.04s/it]

0.7625


In [13]:
count = 0
non_test_list = []
new_query_list = []

for i in tqdm(range(len(pos_df))):
    query = pos_df["query"][i]
    text = pos_df["title"][i]
    if text in new_pos_df[similarity_tensor_pos<0.4]["text"].to_list():
        count += 1
    else:
        USER_PROMPT = f"query: {query}"
        history = [
            SYSTEM_PROMPT,
            USER_PROMPT
        ]
        res = request_llm(history=history)
        try:
            res_query_list = json.loads(res[1]["choices"][0]["message"]["content"].replace("```json","").replace("```","").replace("\n",""))
            non_test_list.append(text)
            new_query_list.append(res_query_list)
        except Exception as e:
            print(e)

        
print(count/len(pos_df))
llm_texts = []
llm_new_queries = []
for i in range(len(new_query_list)):
    for q in new_query_list[i]:
        q = q.replace("query:", "").strip()
        llm_new_queries.append(q)
        llm_texts.append(non_test_list[i])
llm_pos = pd.DataFrame({
    "query":llm_new_queries,
    "text":llm_texts,
    "label":["yes"]*len(llm_texts)
})

100%|██████████| 93/93 [01:44<00:00,  1.12s/it]

0.3225806451612903


In [14]:
similarity_tensor_pos_llm = compute_similarity(llm_pos)
similarity_tensor_neg_llm = compute_similarity(llm_neg)

In [15]:
llm_neg["score"] = similarity_tensor_neg_llm
llm_pos["score"] = similarity_tensor_pos_llm

In [16]:
neg_full = pd.concat([llm_neg[similarity_tensor_neg_llm<0.4],new_neg_df[similarity_tensor_neg<0.4]])
pos_full = pd.concat([llm_pos[similarity_tensor_pos_llm>0.6],new_pos_df[similarity_tensor_pos>0.6]])
neg_full[["query", "text", "label"]].to_csv(f"data/cloud_share/qabot_relevance_data/qwen3_reranker/active_learning/{DATE}/negative_sample.tsv", sep='\t', index=False)
pos_full[["query", "text", "label"]].to_csv(f"data/cloud_share/qabot_relevance_data/qwen3_reranker/active_learning/{DATE}/postive_sample.tsv", sep='\t', index=False)

废弃

In [21]:
# # Call cockpit qabot service
# import time
# from utils.search_utils.call_cockpit_qabot import request_qabot
# neg_query_list = neg_df["query"].to_list()

# query_list = []
# text_list = []
# score_list = []
# for neg_query in neg_query_list:
#     response = request_qabot(neg_query, url="http://cockpit-qabot-lids-testtwo.ssai-apis-staging.chj.cloud:80/expert/query")
#     candi_list = [i["expandQuestion"].strip() for i in response["data"]][:3]
#     score = [i["score"].strip() for i in response["data"]][:3]
    
#     query_list += [neg_query] * len(candi_list)
#     text_list += candi_list
#     score_list += score
#     time.sleep(1)
# neg_aug_df= pd.DataFrame({
#     "query":query_list,
#     "text":text_list,
#     "score":score_list
# })

In [22]:
# pos_query_list = pos_df["query"].to_list()

# query_list = []
# text_list = []
# score_list = []
# for pos_query in pos_query_list:
#     response = request_qabot(pos_query, url="http://cockpit-qabot-lids-testtwo.ssai-apis-staging.chj.cloud:80/expert/query")
#     candi_list = [i["expandQuestion"].strip() for i in response["data"]][:3]
#     score = [i["score"].strip() for i in response["data"]][:3]
    
#     query_list += [pos_query] * len(candi_list)
#     text_list += candi_list
#     score_list += score
#     time.sleep(1)
# pos_aug_df= pd.DataFrame({
#     "query":query_list,
#     "text":text_list,
#     "score":score_list
# })

In [23]:
neg_aug_df.to_csv("data/cloud_share/qabot_relevance_data/qwen3_reranker/active_learning/negative_sample.tsv", sep='\t', index=False)
pos_aug_df.to_csv("data/cloud_share/qabot_relevance_data/qwen3_reranker/active_learning/postive_sample.tsv", sep='\t', index=False)

NameError: name 'neg_aug_df' is not defined